In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, count, last, lag, row_number, substring_index, input_file_name, regexp_extract, expr, concat_ws, map_from_arrays
from pyspark.sql.window import Window

In [2]:
sp = SparkSession.builder.getOrCreate()

24/03/24 22:00:06 WARN Utils: Your hostname, kWs-MacBook-Pro-2019.local resolves to a loopback address: 127.0.0.1; using 192.168.1.205 instead (on interface en0)
24/03/24 22:00:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/24 22:00:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [42]:
src_df = sp.read\
   .option("escapeQuotes", "true")\
   .option("multiLine", "true")\
   .csv(
      'csv/2020-03-29 Coronavirus Tweets.CSV', 
      header=True,
      quote='"',
      escape='\"',
   )\
   .withColumn(
      "tweet_date", 
      regexp_extract(input_file_name(), "/(\d{4}-\d{2}-\d{2})", 1)
   )\
   .withColumn('hashtags', expr(r"regexp_extract_all(text, '(#\\w+)', 0)") )

src_df.show()

+-------------------+-------------------+--------------------+---------------+--------------------+--------------------+------------------+----------------+--------------------+--------+----------+----------------+-------------+------------+---------------+----------+---------------+-------------+------------+--------------------+--------+----+----------+--------------------+
|          status_id|            user_id|          created_at|    screen_name|                text|              source|reply_to_status_id|reply_to_user_id|reply_to_screen_name|is_quote|is_retweet|favourites_count|retweet_count|country_code|place_full_name|place_type|followers_count|friends_count|account_lang|  account_created_at|verified|lang|tweet_date|            hashtags|
+-------------------+-------------------+--------------------+---------------+--------------------+--------------------+------------------+----------------+--------------------+--------+----------+----------------+-------------+------------+-

In [43]:
# q1

derived_df1 = src_df.select(
   "tweet_date",
   "user_id",
   "hashtags",
   "source",
   "screen_name",
   "created_at",
)

derived_df11 = derived_df1\
.select(
   "tweet_date",
   "user_id",
   "source"
)\
.groupBy(
   "tweet_date",
   "user_id"
   # "screen_name_last",
).agg(
   count("*").alias("num_tweets"),
   collect_list(col("source")).alias("source_list")
   # collect_list("hashtags").alias("hashtags_list")
)

# derived_df11 = derived_df11\
#    .withColumn(
#       "source_map", 
#       map_from_arrays(col("source_list"), count(col("source_list")))
# )

derived_df11.show()